In [13]:
import pandas as pd
import numpy as np 
import os
from datetime import datetime

RAW_DATA_PATH = "data/raw/dirty_data.csv"
CLEAN_DATA_PATH = "data/clean/clean_data.csv"
LOG_PATH = "logs/audit_logs.txt"

def log_step(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # Write to file (append mode 'a')
    with open(LOG_PATH, 'a') as f:
        f.write(f"[{timestamp}] {message}\n")
    # Print to screen
    print(f"[{timestamp}] {message}")

print("Setup Complete! Logger is ready.")

Setup Complete! Logger is ready.


In [14]:
# Check current location
print("Current Working Directory:", os.getcwd())

# Go up one level if we are in 'notebooks'
if os.getcwd().endswith("notebooks"):
    os.chdir("..")
    print("Changed directory to:", os.getcwd())

# Now your original paths will work!
RAW_DATA_PATH = "data/raw/dirty_data.csv"


Current Working Directory: /Users/prernaarya/Desktop/automated-data-quality-pipeline


In [15]:
# Check if file exists first
if not os.path.exists(RAW_DATA_PATH):
    print(f"ERROR: The file {RAW_DATA_PATH} does not exist. Please check your folder structure.")
else:
    df = pd.read_csv(RAW_DATA_PATH)
    initial_rows = len(df)
    log_step(f"PIPELINE STARTED. Loaded {initial_rows} rows.")
    
    # Show the first five rows to see how messy it is
    display(df.head())

[2025-12-23 17:33:57] PIPELINE STARTED. Loaded 1000000 rows.


,Transaction_ID,Date,Customer_Name,Product,Total_Items,Total_Cost,Payment_Method,City,Store_Type,Discount_Applied,Customer_Category,Season,Promotion
0,1000000000,2022-01-21 06:27:29,Stacey Price,"['Ketchup', 'Shaving Cream', 'Light Bulbs']",3,71.65,Mobile Payment,Los Angeles,Warehouse Club,True,Homemaker,Winter,NaN
1,1000000001,2023-03-01 13:01:21,Michelle Carlson,"['Ice Cream', 'Milk', 'Olive Oil', 'Bread', 'P...",2,25.93,Cash,San Francisco,Specialty Store,True,Professional,Fall,BOGO (Buy One Get One)
2,1000000002,2024-03-21 15:37:04,Lisa Graves,['Spinach'],6,41.49,Credit Card,Houston,Department Store,True,Professional,Winter,NaN
3,1000000003,2020-10-31 09:59:47,Mrs. Patricia May,"['Tissues', 'Mustard']",1,39.34,Mobile Payment,Chicago,Pharmacy,True,Homemaker,Spring,NaN
4,1000000004,2020-12-10 00:59:59,Susan Mitchell,['Dish Soap'],10,16.42,Debit Card,Houston,Specialty Store,False,Young Adult,Winter,Discount on Selected Items


In [16]:
# 1. Remove Duplicates
duplicates = df.duplicated().sum()
df = df.drop_duplicates()
log_step(f"Removed {duplicates} duplicate rows.")

# 2. Standardize Column Names
# Before: "Last Name", "Phone/Number"
# After: "last_name", "phone_number"
df.columns = [x.lower().strip().replace(" ", "_").replace("/", "_") for x in df.columns]
log_step("Standardized column names to snake_case.")

# Check the new column names
print(df.columns)

[2025-12-23 17:33:58] Removed 0 duplicate rows.
[2025-12-23 17:33:58] Standardized column names to snake_case.
Index(['transaction_id', 'date', 'customer_name', 'product', 'total_items',
       'total_cost', 'payment_method', 'city', 'store_type',
       'discount_applied', 'customer_category', 'season', 'promotion'],
      dtype='object')


In [17]:
# Check for missing values in Promotion
if 'promotion' in df.columns:
    missing_count = df['promotion'].isnull().sum()
    if missing_count > 0:
        df['promotion'] = df['promotion'].fillna("None")
        log_step(f"Imputed {missing_count} missing 'promotion' values with 'None'.")
    else:
        log_step("No missing values found in 'promotion'.")

# Check result
display(df['promotion'].value_counts(dropna=False).head())

[2025-12-23 17:33:58] Imputed 333943 missing 'promotion' values with 'None'.


promotion
None                          333943
Discount on Selected Items    333370
BOGO (Buy One Get One)        332687
Name: count, dtype: int64

In [18]:
# Convert 'date' column to datetime objects
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    log_step("Converted 'date' column to datetime objects.")
    
    # Validation: Check if any dates failed to convert
    invalid_dates = df['date'].isnull().sum()
    if invalid_dates > 0:
        log_step(f"WARNING: {invalid_dates} rows have invalid dates and were set to NaT.")
    
    # Show the data type to confirm
    print(df['date'].dtype)

[2025-12-23 17:33:58] Converted 'date' column to datetime objects.
datetime64[ns]


In [19]:
# Remove brackets and quotes from Product column
if 'product' in df.columns:
    # Use Regex to replace [ ] and ' with nothing
    df['product'] = df['product'].astype(str).str.replace(r"[\[\]']", "", regex=True)
    log_step("Cleaned special characters (brackets/quotes) from 'product' column.")
    
    display(df[['product']].head())

[2025-12-23 17:34:00] Cleaned special characters (brackets/quotes) from 'product' column.


,product
0,"Ketchup, Shaving Cream, Light Bulbs"
1,"Ice Cream, Milk, Olive Oil, Bread, Potatoes"
2,Spinach
3,"Tissues, Mustard"
4,Dish Soap


In [20]:
display(df.head())

,transaction_id,date,customer_name,product,total_items,total_cost,payment_method,city,store_type,discount_applied,customer_category,season,promotion
0,1000000000,2022-01-21 06:27:29,Stacey Price,"Ketchup, Shaving Cream, Light Bulbs",3,71.65,Mobile Payment,Los Angeles,Warehouse Club,True,Homemaker,Winter,None
1,1000000001,2023-03-01 13:01:21,Michelle Carlson,"Ice Cream, Milk, Olive Oil, Bread, Potatoes",2,25.93,Cash,San Francisco,Specialty Store,True,Professional,Fall,BOGO (Buy One Get One)
2,1000000002,2024-03-21 15:37:04,Lisa Graves,Spinach,6,41.49,Credit Card,Houston,Department Store,True,Professional,Winter,None
3,1000000003,2020-10-31 09:59:47,Mrs. Patricia May,"Tissues, Mustard",1,39.34,Mobile Payment,Chicago,Pharmacy,True,Homemaker,Spring,None
4,1000000004,2020-12-10 00:59:59,Susan Mitchell,Dish Soap,10,16.42,Debit Card,Houston,Specialty Store,False,Young Adult,Winter,Discount on Selected Items
